# Automatiser un traitement Excel avec Python
## Exploration de la librairie openpyxl

#### Meyssa Beddar

L'objectif de ce notebook est de fournir un exemple d'automatisation qui pourrait être réalisé avec les données du brfss.

Plus précisément, le brfss (le système de surveillance des facteurs de risque comportemental en français) est le premier système national d'enquêtes téléphoniques liées à la santé qui collectent des données d'état sur les résidents des États-Unis concernant leurs comportements à risque liés à la santé, leurs problèmes de santé chroniques et l'utilisation des services de prévention.
Ce dernier étant réalisé chaque année, il pourrait être intéressant de réaliser un fichier excel annuel comprenant les informations globales ayant été collectées. 

Etant donné le nombre très important de colonnes du dataset initial (330 environ par année) et le caractère illustratif de l'exercice, je n'ai retenu que 11 colonnes pour la réalisation de ce rapport reprenant des informations sur la socio-démographie, le tabagisme et la santé mentale des répondants. Pour les mêmes raisons, je n'ai collecté que les jeux de données des années 2018 et 2019.

Enfin, la passation du questionnaire étant auprès de 400 000 individus chaque année, un échantillon de 50 000 personnes par année sera créé afin de limiter les temps de chargement. 

# Importation des librairies et du jeu de données

### Chargement des librairies


In [86]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.chart import (
    PieChart,
    ProjectedPieChart,
    Reference
)
from openpyxl.chart.series import DataPoint

from openpyxl.chart.label import DataLabelList 

from openpyxl.chart import Reference, BarChart3D

### Chargement du  jeu de données

In [2]:
df = pd.read_csv("data_2018_2019.csv",encoding='latin-1')

### Affichage des 5 premières lignes

In [3]:
df.head()

,IYEAR,MARITAL,SEX,MENTHLTH_FACTOR,POORHEALTH_FACTOR,X_RFSMOK3_FACTOR,ADDEPEV2_FACTOR,X_AGE_G_FACTOR,X_EDUCAG_FACTOR,X_INCOMG_FACTOR,SMOKDAY2_FACTOR
0,2018,Divorcé,Femme,Santé ment. +,Santé ment/phy +,Non,Pas de dépression.,55-64 ans,A étudié à l'univ.,< 15 000,NaN
1,2018,Veuf,Homme,Santé ment. +,Santé ment/phy +,Non,Dépression,65ans et plus,A étudié à l'univ.,35 - 50 000,NaN
2,2018,Jamais marié,Femme,Santé ment. +,Santé ment/phy +,Non,Pas de dépression.,45-54 ans,Sans diplôme,NaN,NaN
3,2018,Veuf,Femme,Santé ment. +,Santé ment/phy +,Non,Dépression,65ans et plus,Sans diplôme,15 - 25 000,Jamais
4,2018,Jamais marié,Homme,Santé ment. +,Santé ment/phy +,Non,Dépression,55-64 ans,A étudié à l'univ.,15 - 25 000,NaN


Les différentes colonnes retenues sont :
- IYEAR	: l'année (2018 ou 2019)

- MARITAL : Statut matrimonial (Jamais marié, Marié, Divorcé, Membre d'un couple non marié, Veuf, Séparé)
- SEX : Sexe (Homme ou Femme)
- X_AGE_G_FACTOR : Age (18-24 ans, 25-34 ans, 35-44 ans, 45-54 ans, 55-64 ans, 65ans et plus)
- X_EDUCAG_FACTOR : Niveau d'éducation (Sans diplôme, Diplôme du lycée, A étudié à l'univ., Diplôme de l'université)
- X_INCOMG_FACTOR : Revenus (< 15 000 , 15 - 25 000, 25 - 35 000, 35 - 50 000, > 50 000)

- MENTHLTH_FACTOR : Ressenti par rapport à la santé mentale sur les 30 derniers jours (Santé ment. -, Santé ment. +-, Santé ment. +)
- POORHEALTH_FACTOR	: Ressenti par rapport à la santé globale sur les 30 derniers jours (Santé ment/phy -, Santé ment/phy +-, Santé ment/phy +)
- ADDEPEV2_FACTOR : Présence de dépression (Pas de dépression, Dépression)

- X_RFSMOK3_FACTOR : Fumeur (Oui, Non)

# Rapide nettoyage des données

### Renommage de deux modalités de la colonne "X_INCOMG_FACTOR"

In [4]:
df['X_INCOMG_FACTOR'] = df['X_INCOMG_FACTOR'].replace(['< 15 000'],'15 000 et moins')

In [5]:
df['X_INCOMG_FACTOR'] = df['X_INCOMG_FACTOR'].replace(['< 50 000'],'50 000 et plus')

In [6]:
df['X_INCOMG_FACTOR'].unique()

array(['15 000 et moins', '35 - 50 000', nan, '15 - 25 000',
       '25 - 35 000', '50 000 et plus'], dtype=object)

# Création des fichiers excels par année

#### Création d'une liste reprenant les années présentes dans le jeu de données initial

In [70]:
list_year = list(df['IYEAR'].unique())
list_year

list

#### Création d'un fichier excel par année et importation des données annuelles dans la feuille "Data"

In [87]:
for i in list_year:
    #Création d'un dataframe par année
    df_filtered = df[df['IYEAR'] == i]
    # echantillonage pour réduire les temps de chargement du projet, autrement ça n'est pas nécessaire
    df_filtered = df_filtered.sample(50000)
    #Création du fichier excel
    filename = str(i) + '.xlsx'
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    df_filtered.to_excel(writer, sheet_name='Data')
    writer.save()

#### Définition de styles pour les tableaux

In [ ]:
#Style bleu clair
lightStyle = openpyxl.worksheet.table.TableStyleInfo(name='TableStyleLight2',
                                                      showRowStripes=True)
#Style bleu
mediumStyle = openpyxl.worksheet.table.TableStyleInfo(name='TableStyleMedium2',
                                                      showRowStripes=True)

#### Mise en forme de la feuille "Data" en tableau

In [88]:
for i in list_year:
    #Chargement des fichiers excels
    filename = str(i) + '.xlsx'
    workbook = load_workbook(filename=filename)
    worksheet = workbook['Data']
    
    # Création d'un tableau dans la feuille "Data"
    table = openpyxl.worksheet.table.Table(ref='A1:L50001',
                                       displayName='Données',
                                       tableStyleInfo=mediumStyle)
    worksheet.add_table(table)
    
    
    workbook.save(filename=filename)

C:\Users\meyss\anaconda3\lib\site-packages\openpyxl\worksheet\_writer.py:273: UserWarning: File may not be readable: column headings must be strings.
  warn("File may not be readable: column headings must be strings.")


# Création des feuilles "DSD", "Tabagisme" et "Santé"

### Création des feuilles  :

- "DSD" : Données sociodémographiques des répondants
- "Tabagisme" : Fumeur / non fumeur des répondants
- "Santé" : Santé mentale des répondants

Les trois feuilles "DSD", "Tabagisme" et "Santé" reprendront respectivement les informations générales annuelles des répondants concernant:
- l'âge, le sexe, les revenus, les niveaux d'éducation et le statut matrimonial;
- le fait d'être fumeur ou non fumeur;
- le ressenti sur la santé globale, la santé mentale et la présence de dépression. 

In [89]:
for i in list_year:
    #Chargement des fichiers excels
    filename = str(i) + '.xlsx'
    workbook = load_workbook(filename=filename)
    worksheet = workbook['Data']
    #Création des trois feuilles
    sheet2 = workbook.create_sheet("DSD", 2) 
    sheet3 = workbook.create_sheet("Tabagisme", 3) 
    sheet4 = workbook.create_sheet("Santé", 4) 
    workbook.sheetnames
    #Enregistrement des modifications
    workbook.save(filename=filename)

# Insertion des tableaux et graphiques récapitulatifs dans la feuille "DSD"

### Remplissage de la feuille "DSD"

In [90]:
for i in list_year:
    #Chargement des fichiers excels
    filename = str(i) + '.xlsx'
    workbook = load_workbook(filename=filename)
    sheet2 = workbook['DSD']

#### Répartition du sexe des répondants    
    #Titres
    sheet2['A1'] = 'Sexe'
    sheet2['B1'] = 'Nb de répondants'
    sheet2['C1'] = 'Pourcentage'


    sheet2['A2'] = 'Femme'
    sheet2['A3'] = 'Homme'""
    sheet2['A4'] = 'Total'
    
    #Comptage des modalités
    
    sheet2['B2'] = "=COUNTIFS(Data!D:D,A2)"
    sheet2['B3'] = "=COUNTIFS(Data!D:D,A3)"
    sheet2['B4'] = "=SUM(B2,B3)"
    
    #Calcul des pourcentages des modalités

    sheet2['C2'] = "=B2/B4"
    sheet2['C3'] = "=B3/B4"

    sheet2['C2'].number_format = '0.00%'
    sheet2['C3'].number_format = '0.00%'

    worksheet = workbook['DSD']

    # Création d'un tableau sur ces données
    table = openpyxl.worksheet.table.Table(ref='A1:C4',
                                       displayName='Sexe',
                                       tableStyleInfo=lightStyle)
    worksheet.add_table(table)
    
    
#### Répartition des statuts matrimoniaux des répondants     
    
    sheet2['A7'] = 'Statut matrimonial'
    sheet2['B7'] = 'Nb de répondants'
    sheet2['C7'] = 'Pourcentage'

    sheet2['A8'] = 'Jamais marié'
    sheet2['A9'] = 'Marié'
    sheet2['A10'] = 'Divorcé'
    sheet2['A11'] = "Membre d'un couple non marié"
    sheet2['A12'] = 'Veuf'
    sheet2['A13'] = 'Séparé'
    sheet2['A14'] = 'Total'

    sheet2['B8'] = "=COUNTIFS(Data!C:C,A8)"
    sheet2['B9'] = "=COUNTIFS(Data!C:C,A9)"
    sheet2['B10'] = "=COUNTIFS(Data!C:C,A10)"
    sheet2['B11'] = "=COUNTIFS(Data!C:C,A11)"
    sheet2['B12'] = "=COUNTIFS(Data!C:C,A12)"
    sheet2['B13'] = "=COUNTIFS(Data!C:C,A13)"
    sheet2['B14'] = "=SUM(B8,B9,B10,B11,B12,B13)"


    sheet2['C8'] = "=B8/B14"
    sheet2['C9'] = "=B9/B14"
    sheet2['C10'] = "=B10/B14"
    sheet2['C11'] = "=B11/B14"
    sheet2['C12'] = "=B12/B14"
    sheet2['C13'] = "=B13/B14"

    sheet2['C8'].number_format = '0.00%'
    sheet2['C9'].number_format = '0.00%'
    sheet2['C10'].number_format = '0.00%'
    sheet2['C11'].number_format = '0.00%'
    sheet2['C12'].number_format = '0.00%'
    sheet2['C13'].number_format = '0.00%'


    # Création du tableau
    table = openpyxl.worksheet.table.Table(ref='A7:C14',
                                       displayName='Marital',
                                       tableStyleInfo=lightStyle)
    worksheet.add_table(table)
    

#### Répartition des classes d'âge des répondants 
    
    sheet2['A17'] = 'Age'
    sheet2['B17'] = 'Nb de répondants'
    sheet2['C17'] = 'Pourcentage'

    sheet2['A18'] = '18-24 ans'
    sheet2['A19'] = "25-34 ans"
    sheet2['A20'] = '35-44 ans'
    sheet2['A21'] = "45-54 ans"
    sheet2['A22'] = '55-64 ans'
    sheet2['A23'] = '65ans et plus'
    sheet2['A24'] = 'Total'

    sheet2['B18'] = "=COUNTIFS(Data!I:I,A18)"
    sheet2['B19'] = "=COUNTIFS(Data!I:I,A19)"
    sheet2['B20'] = "=COUNTIFS(Data!I:I,A20)"
    sheet2['B21'] = "=COUNTIFS(Data!I:I,A21)"
    sheet2['B22'] = "=COUNTIFS(Data!I:I,A22)"
    sheet2['B23'] = "=COUNTIFS(Data!I:I,A23)"
    sheet2['B24'] = "=SUM(B18,B19,B20,B21,B22,B23)"

    sheet2['C18'] = "=B18/B24"
    sheet2['C19'] = "=B19/B24"
    sheet2['C20'] = "=B20/B24"
    sheet2['C21'] = "=B21/B24"
    sheet2['C22'] = "=B22/B24"
    sheet2['C23'] = "=B23/B24"

    sheet2['C18'].number_format = '0.00%'
    sheet2['C19'].number_format = '0.00%'
    sheet2['C20'].number_format = '0.00%'
    sheet2['C21'].number_format = '0.00%'
    sheet2['C22'].number_format = '0.00%'
    sheet2['C23'].number_format = '0.00%'

    # Création d'un tableau
    table = openpyxl.worksheet.table.Table(ref='A17:C24',
                                       displayName='Age',
                                       tableStyleInfo=lightStyle)
    worksheet.add_table(table)

    
#### Répartition des niveaux d'éducation des répondants 
    
    sheet2['F1'] = "Niveau d'éducation"
    sheet2['G1'] = 'Nb de répondants'
    sheet2['H1'] = 'Pourcentage'

    sheet2['F2'] = 'Sans diplôme'
    sheet2['F3'] = "Diplôme du lycée"
    sheet2['F4'] = "A étudié à l'univ."
    sheet2['F5'] = "Diplôme de l'université"
    sheet2['F6'] = 'Total'

    sheet2['G2'] = "=COUNTIFS(Data!J:J,F2)"
    sheet2['G3'] = "=COUNTIFS(Data!J:J,F3)"
    sheet2['G4'] = "=COUNTIFS(Data!J:J,F4)"
    sheet2['G5'] = "=COUNTIFS(Data!J:J,F5)"
    sheet2['G6'] = "=SUM(G2,G3,G4,G5)"

    sheet2['H2'] = "=G2/G6"
    sheet2['H3'] = "=G3/G6"
    sheet2['H4'] = "=G4/G6"
    sheet2['H5'] = "=G5/G6"

    sheet2['H2'].number_format = '0.00%'
    sheet2['H3'].number_format = '0.00%'
    sheet2['H4'].number_format = '0.00%'
    sheet2['H5'].number_format = '0.00%'

    # Création d'un tableau
    table = openpyxl.worksheet.table.Table(ref='F1:H6',
                                       displayName='Etudes',
                                       tableStyleInfo=lightStyle)
    worksheet.add_table(table)
    

#### Répartition des niveaux de revenus des répondants     
    
    sheet2['F9'] = "Niveau de revenus"
    sheet2['G9'] = 'Nb de répondants'
    sheet2['H9'] = 'Pourcentage'

    sheet2['F10'] = '15 000 et moins'
    sheet2['F11'] = "15 - 25 000"
    sheet2['F12'] = "25 - 35 000"
    sheet2['F13'] = "35 - 50 000"
    sheet2['F14'] = "50 000 et plus"
    sheet2['F15'] = 'Total'

    sheet2['G10'] = "=COUNTIFS(Data!K:K,F10)"
    sheet2['G11'] = "=COUNTIFS(Data!K:K,F11)"
    sheet2['G12'] = "=COUNTIFS(Data!K:K,F12)"
    sheet2['G13'] = "=COUNTIFS(Data!K:K,F13)"
    sheet2['G14'] = "=COUNTIFS(Data!K:K,F14)"
    sheet2['G15'] = "=SUM(G10,G11,G12,G13,G14)"

    sheet2['H10'] = "=G10/G15"
    sheet2['H11'] = "=G11/G15"
    sheet2['H12'] = "=G12/G15"
    sheet2['H13'] = "=G13/G15"
    sheet2['H14'] = "=G14/G15"

    sheet2['H10'].number_format = '0.00%'
    sheet2['H11'].number_format = '0.00%'
    sheet2['H12'].number_format = '0.00%'
    sheet2['H13'].number_format = '0.00%'
    sheet2['H14'].number_format = '0.00%'

    # Création d'un tableau
    table = openpyxl.worksheet.table.Table(ref='F9:H15',
                                       displayName='Revenus',
                                       tableStyleInfo=lightStyle)
    worksheet.add_table(table)
    

#### Enregistrement des modifications
    
    workbook.save(filename=filename)

### Ajout des graphiques dans la feuille "DSD"

In [91]:
for i in list_year:
    filename = str(i) + '.xlsx'
    workbook = load_workbook(filename=filename)
    sheet2 = workbook['DSD']
    
#### Répartition du sexe des répondants - Camembert   
    
    pie = PieChart()
    # Labels
    labels = Reference(sheet2, min_col=1, min_row=2, max_row=3)
    # Données
    data = Reference(sheet2, min_col=2, min_row=1, max_row=3)
    pie.add_data(data, titles_from_data=True)
    pie.set_categories(labels)
    # Titre
    pie.title = "Sexe des répondants"

    pie.dataLabels = DataLabelList() 
    pie.dataLabels.showPercent = True 

    sheet2.add_chart(pie, "J2")

    
#### Répartition du statut matrimonial des répondants - Diagramme en bar 3D   

    #Récupération des valeurs
    values = Reference(sheet2,
                   min_col=2,
                   max_col=2,
                   min_row=7,
                   max_row=13)

    cats = Reference(sheet2, min_col=1, max_col=1, min_row=8, max_row=14)


    chart = BarChart3D()

    chart.add_data(values, titles_from_data=True)
    chart.set_categories(cats)

    chart.dataLabels = DataLabelList() 
    chart.dataLabels.showPercent = True 

    chart.title = "Statut matrimonial des répondants"

    chart.x_axis.title = "Statut matrimonial"

    chart.y_axis.title = "Nombre de répondants"

    sheet2.add_chart(chart,"B28")
    
#### Répartition du niveau d'éducation des répondants       
    
    
    values = Reference(sheet2,
                   min_col=7,
                   max_col=7,
                   min_row=1,
                   max_row=5)

    cats = Reference(sheet2, min_col=6, max_col=6, min_row=2, max_row=6)

    chart = BarChart3D()

    chart.add_data(values, titles_from_data=True)
    chart.set_categories(cats)

    chart.dataLabels = DataLabelList() 
    chart.dataLabels.showPercent = True 

    chart.title = "Niveau d'éducation des répondants"

    chart.x_axis.title = "Niveau d'éducation"

    chart.y_axis.title = "Nombre de répondants"

    sheet2.add_chart(chart,"N28")
    
    
#### Répartition des revenus des répondants      
    
    
    values = Reference(sheet2,
                   min_col=7,
                   max_col=7,
                   min_row=9,
                   max_row=14)

    cats = Reference(sheet2, min_col=6, max_col=6, min_row=10, max_row=15)

    chart = BarChart3D()

    chart.add_data(values, titles_from_data=True)
    chart.set_categories(cats)

    chart.dataLabels = DataLabelList() 
    chart.dataLabels.showPercent = True 

    chart.title = "Revenus des répondants"

    chart.x_axis.title = "Niveau de revenus"

    chart.y_axis.title = "Nombre de répondants"

    sheet2.add_chart(chart,"B46")
    
    
#### Répartition de l'âge des répondants       
    
    
    values = Reference(sheet2,
                   min_col=2,
                   max_col=2,
                   min_row=17,
                   max_row=23)

    cats = Reference(sheet2, min_col=1, max_col=1, min_row=18, max_row=24)

    chart = BarChart3D()

    chart.add_data(values, titles_from_data=True)
    chart.set_categories(cats)

    chart.dataLabels = DataLabelList() 
    chart.dataLabels.showPercent = True 

    chart.title = "Âge des répondants"

    chart.x_axis.title = "Classes d'âge"

    chart.y_axis.title = "Nombre de répondants"


    sheet2.add_chart(chart,"N46")
    
    
#### Enregistrement des modifications   
    
    workbook.save(filename=filename)

# Insertion des tableaux et graphiques récapitulatifs dans la feuille "Tabagisme"

### Ajout du tableau relatif au tabagisme dans la feuille "Tabagisme"

In [92]:
for i in list_year:
    filename = str(i) + '.xlsx'
    workbook = load_workbook(filename=filename)
    sheet3 = workbook['Tabagisme']


#### Répartition des fumeurs parmi les répondants   

    sheet3['A1'] = 'Fumeur'
    sheet3['B1'] = 'Nb de répondants'
    sheet3['C1'] = 'Pourcentage'

    sheet3['A2'] = 'Oui'
    sheet3['A3'] = 'Non'
    sheet3['A4'] = 'Total'

    sheet3['B2'] = "=COUNTIFS(Data!G:G,A2)"
    sheet3['B3'] = "=COUNTIFS(Data!G:G,A3)"
    sheet3['B4'] = "=SUM(B2,B3)"


    sheet3['C2'] = "=B2/B4"
    sheet3['C3'] = "=B3/B4"

    sheet3['C2'].number_format = '0.00%'
    sheet3['C3'].number_format = '0.00%'


    worksheet = workbook['Tabagisme']

    # Création d'un tableau
    table = openpyxl.worksheet.table.Table(ref='A1:C4',
                                       displayName='Tabac',
                                       tableStyleInfo=lightStyle)
    worksheet.add_table(table)
    

#### Enregistrement des modifications

    workbook.save(filename=filename)

### Ajout du graphique dans la feuille "Tabagisme"

In [93]:
for i in list_year:
    filename = str(i) + '.xlsx'
    workbook = load_workbook(filename=filename)
    sheet3 = workbook['Tabagisme']


#### Répartition des fumeurs parmi les répondants  - Camembert 

    pie = PieChart()
    labels = Reference(sheet3, min_col=1, min_row=2, max_row=3)
    data = Reference(sheet3, min_col=2, min_row=1, max_row=3)
    pie.add_data(data, titles_from_data=True)
    pie.set_categories(labels)
    pie.title = "Tabagisme des répondants"

    pie.dataLabels = DataLabelList() 
    pie.dataLabels.showPercent = True 

    sheet3.add_chart(pie, "J2")

    
#### Enregistrement des modifications
    
    workbook.save(filename=filename)

# Insertion des tableaux et graphiques récapitulatifs dans la feuille "Santé"

### Ajout des tableaux dans la feuille "Santé"

In [94]:
for i in list_year:
    filename = str(i) + '.xlsx'
    workbook = load_workbook(filename=filename)
    sheet4 = workbook['Santé']

    
#### Ressenti par rapport à la santé globale des répondants
    
    sheet4['A1'] = 'Santé physique et mentale des répondants'
    sheet4['B1'] = 'Nb de répondants'
    sheet4['C1'] = 'Pourcentage'

    sheet4['A2'] = 'Santé ment/phy -'
    sheet4['A3'] = 'Santé ment/phy +-'
    sheet4['A4'] = 'Santé ment/phy +'
    sheet4['A5'] = 'Total'

    sheet4['B2'] = "=COUNTIFS(Data!F:F,A2)"
    sheet4['B3'] = "=COUNTIFS(Data!F:F,A3)"
    sheet4['B4'] = "=COUNTIFS(Data!F:F,A4)"
    sheet4['B5'] = "=SUM(B2,B3,B4)"

    sheet4['C2'] = "=B2/B5"
    sheet4['C3'] = "=B3/B5"
    sheet4['C4'] = "=B4/B5"

    sheet4['C2'].number_format = '0.00%'
    sheet4['C3'].number_format = '0.00%'
    sheet4['C4'].number_format = '0.00%'

    worksheet = workbook['Santé']

    # Création du tableau
    table = openpyxl.worksheet.table.Table(ref='A1:C5',
                                       displayName='Mtle/phy',
                                       tableStyleInfo=lightStyle)
    worksheet.add_table(table)
    
    
#### Ressenti par rapport à la santé mentale des répondants    
    
    
    sheet4['E1'] = 'Santé mentale des répondants'
    sheet4['F1'] = 'Nb de répondants'
    sheet4['G1'] = 'Pourcentage'

    sheet4['E2'] = 'Santé ment. -'
    sheet4['E3'] = 'Santé ment. +-'
    sheet4['E4'] = 'Santé ment. +'
    sheet4['E5'] = 'Total'

    sheet4['F2'] = "=COUNTIFS(Data!E:E,E2)"
    sheet4['F3'] = "=COUNTIFS(Data!E:E,E3)"
    sheet4['F4'] = "=COUNTIFS(Data!E:E,E4)"
    sheet4['F5'] = "=SUM(F2,F3,F4)"

    sheet4['G2'] = "=F2/F5"
    sheet4['G3'] = "=F3/F5"
    sheet4['G4'] = "=F4/F5"

    sheet4['G2'].number_format = '0.00%'
    sheet4['G3'].number_format = '0.00%'
    sheet4['G4'].number_format = '0.00%'


    # Création d'un tableau
    table = openpyxl.worksheet.table.Table(ref='E1:G5',
                                       displayName='Mtle',
                                       tableStyleInfo=lightStyle)
    worksheet.add_table(table)
    
    
    
#### Prévalence de la dépression parmi les répondants    
    
    sheet4['I1'] = 'Diagnostic de dépression'
    sheet4['J1'] = 'Nb de répondants'
    sheet4['K1'] = 'Pourcentage'

    sheet4['I2'] = 'Dépression'
    sheet4['I3'] = 'Pas de dépression.'
    sheet4['I4'] = 'Total'

    sheet4['J2'] = "=COUNTIFS(Data!H:H,I2)"
    sheet4['J3'] = "=COUNTIFS(Data!H:H,I3)"
    sheet4['J4'] = "=SUM(J2,J3)"

    sheet4['K2'] = "=J2/J4"
    sheet4['K3'] = "=J3/J4"

    sheet4['K2'].number_format = '0.00%'
    sheet4['K3'].number_format = '0.00%'

    # Création du tableau
    table = openpyxl.worksheet.table.Table(ref='I1:K4',
                                       displayName='Dépression',
                                       tableStyleInfo=lightStyle)
    worksheet.add_table(table)
    
    
    
#### Enregistrement des modifications  
    
    
    workbook.save(filename=filename)

### Ajout des graphiques dans la feuille "Santé"

In [95]:
for i in list_year:
    filename = str(i) + '.xlsx'
    workbook = load_workbook(filename=filename)
    sheet4 = workbook['Santé']
    

#### Ressenti par rapport à la santé globale des répondants
    
    pie = PieChart()
    labels = Reference(sheet4, min_col=9, min_row=2, max_row=3)
    data = Reference(sheet4, min_col=10, min_row=1, max_row=3)
    pie.add_data(data, titles_from_data=True)
    pie.set_categories(labels)
    pie.title = "Dépressivité des répondants"

    pie.dataLabels = DataLabelList() 
    pie.dataLabels.showPercent = True 

    sheet4.add_chart(pie, "F7")
    

#### Ressenti par rapport à la santé mentale des répondants    
    
    pie = PieChart()
    labels = Reference(sheet4, min_col=1, min_row=2, max_row=4)
    data = Reference(sheet4, min_col=2, min_row=1, max_row=4)
    pie.add_data(data, titles_from_data=True)
    pie.set_categories(labels)
    pie.title = "Santé physique et mentale des répondants"

    pie.dataLabels = DataLabelList() 
    pie.dataLabels.showPercent = True 

    # Cut the first slice out of the pie
    slice = DataPoint(idx=0, explosion=20)
    pie.series[0].data_points = [slice]

    sheet4.add_chart(pie, "B22")
    

#### Prévalence de la dépression parmi les répondants    
    
    pie = PieChart()
    labels = Reference(sheet4, min_col=5, min_row=2, max_row=4)
    data = Reference(sheet4, min_col=6, min_row=1, max_row=4)
    pie.add_data(data, titles_from_data=True)
    pie.set_categories(labels)
    pie.title = "Santé mentale des répondants"

    pie.dataLabels = DataLabelList() 
    pie.dataLabels.showPercent = True 

    # Cut the first slice out of the pie
    slice = DataPoint(idx=0, explosion=20)
    pie.series[0].data_points = [slice]

    sheet4.add_chart(pie, "L22")
    

#### Enregistrement des modifications  
    
    workbook.save(filename=filename)